In [ ]:
import pandas as pd
import urllib
import shutil
import cv2
import os
import numpy as np
from keras.preprocessing import image

In [ ]:
def load_dataset(inputpath):
    list_of_tensors = []
    for dirpath, dirnames, filenames in os.walk(inputpath):
        #print("Files in this '" + inputpath + "' dir is " + str(len(filenames)))
        for filename in filenames:
            img_path = os.path.join(dirpath,filename)
            img = image.load_img(img_path, target_size=(224,224))#,grayscale=True) #rescaling images to 224x224
            x = image.img_to_array(img)
            list_of_tensors.append(np.expand_dims(x, axis=0))
    return np.vstack(list_of_tensors)

In [ ]:
x_train = load_dataset('train')
x_test = load_dataset('test')

print(x_train.shape)
print(x_test.shape)

In [ ]:
from keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions

# define ResNet50 model
ResNet50_model = ResNet50(weights='imagenet')

def ResNet50_predict_labels(x_train):
    # returns prediction vector for image located at img_path
    img = preprocess_input(x_train)
    return np.argmax(ResNet50_model.predict(img))

print(ResNet50_predict_labels(x_train))

In [ ]:
#We will normalize all values between 0 and 1 and we will flatten the 224x224x3 images into vectors of size 150528.
x_train_f = x_train.astype('float32')/255 
x_test_f = x_test.astype('float32')/255

# Convert 4D (numOfRows, 224, 224, 3) ==> (224x224x3)
x_train_f = x_train_f.reshape((len(x_train_f), np.prod(x_train_f.shape[1:])))
x_test_f = x_test_f.reshape((len(x_test_f), np.prod(x_test_f.shape[1:])))

print("Training set: ", x_train_f.shape)
print("Testing set: ", x_test_f.shape)

In [ ]:
#Implementing basic CNN from scratch
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential

numOfLabels = 200 #random number guess!! 

cnn = Sequential()
cnn.add(Conv2D(filters=16, kernel_size=2, padding='same', activation='relu', input_shape=(224,224,3)))
cnn.add(MaxPooling2D(pool_size=2))
cnn.add(Conv2D(filters=32, kernel_size=2, padding='same', activation='relu'))
cnn.add(MaxPooling2D(pool_size=2))
cnn.add(Conv2D(filters=64, kernel_size=2, padding='same', activation='relu'))
cnn.add(MaxPooling2D(pool_size=2))
cnn.add(GlobalAveragePooling2D())
cnn.add(Dense(numOfLabels, activation='softmax'))

cnn.summary()

In [ ]:
cnn.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
from keras.callbacks import ModelCheckpoint  

epochs = 10
checkpointer = ModelCheckpoint(filepath='weights.best.from_scratch.hdf5', 
                               verbose=1, save_best_only=True)

cnn.fit(x_train_f, x_train_f, 
          validation_data=(x_test_f, x_test_f),
          epochs=epochs, batch_size=20, callbacks=[checkpointer], verbose=1)

In [ ]:
model.load_weights('weights.best.from_scratch.hdf5')

In [ ]:
# get index of predicted dog breed for each image in test set
dog_breed_predictions = [np.argmax(model.predict(np.expand_dims(tensor, axis=0))) for tensor in test_tensors]

# report test accuracy
test_accuracy = 100*np.sum(np.array(dog_breed_predictions)==np.argmax(test_targets, axis=1))/len(dog_breed_predictions)
print('Test accuracy: %.4f%%' % test_accuracy)